<a href="https://colab.research.google.com/github/BaluDream/1978asdream/blob/main/2022_December_17_fastai_YouTube_closed_captions_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install pytube

In [ ]:
pip install youtube_transcript_api

In [3]:
from youtube_transcript_api import YouTubeTranscriptApi
from pytube import YouTube, Playlist

import sqlite3

In [4]:
base_pl = 'https://www.youtube.com/playlist?list='
base_yt = 'https://youtu.be/'

yt_pl_ids = [
    'PLfYUBJiXbdtSgU6S_3l6pX-4hQYKNJZFU', # fast.ai APL Study Group #2022
    'PLfYUBJiXbdtSvpQjSnJJ_PmDQB_VyT5iU', # Practical Deep Learning for Coders 2022
    'PLfYUBJiXbdtSLBPJ1GMx-sQWf6iNhb8mM', # fast.ai live coding & tutorials #2022
    'PLfYUBJiXbdtRL3FMB3GoWHRI8ieU6FhfM', # Practical Deep Learning for Coders (2020)
    'PLfYUBJiXbdtTIdtE1U8qgyxo4Jy2Y91uj', # Deep Learning from the Foundations #2019
    'PLfYUBJiXbdtSWRCYUHh-ThVCC39bp5yiq', # fastai v2 code walk-thrus #2019
    'PLfYUBJiXbdtSIJb-Qd3pw0cqCbkGeS0xn', # Practical Deep Learning for Coders 2019
    'PLfYUBJiXbdtSyktd8A_x0JNd6lxDcZE96', # Introduction to Machine Learning for Coders
    'PLfYUBJiXbdtTttBGq-u2zeY1OTjs5e-Ia', # Cutting Edge Deep Learning for Coders 2 #2018
    'PLfYUBJiXbdtS2UQRzyrxmyVHoGW0gmLSM', # Practical Deep Learning For Coders 2018
]

In [6]:
playlist = Playlist('https://www.youtube.com/playlist?list=PLfYUBJiXbdtSvpQjSnJJ_PmDQB_VyT5iU')
print(playlist.title)
video = YouTube(playlist[0])
print(video.title)
print(playlist[0])
video_id = playlist[0].split('=')[1]
script = YouTubeTranscriptApi.get_transcript(video_id, languages=('en',))
print(script[0])

Practical Deep Learning for Coders 2022
Lesson 1: Practical Deep Learning for Coders 2022
https://www.youtube.com/watch?v=8SF_h3xF3cE
{'text': 'Welcome to Practical Deep Learning for coders,\xa0\nlesson one. This is version five of this course,\xa0\xa0', 'start': 2.0, 'duration': 8.0}


In [ ]:
playlists = dict()
videos = dict()
database = dict()

for pl_id in yt_pl_ids:
    playlist = Playlist(base_pl + pl_id)
    print(playlist.title)
    playlists[pl_id] = playlist.title
    database[pl_id] = dict()

    for video in playlist:
        video_id = video.split("=")[1]
        videos[video_id] = YouTube(video).title
        database[pl_id][video_id] = dict()
        # Manually created transcripts are returned first
        script = YouTubeTranscriptApi.get_transcript(video_id, languages=('en',))

        for txt in script:
            database[pl_id][video_id][txt['start']] = txt['text']

Building Database

In [8]:
# https://stackoverflow.com/a/60932565/10013187
records = [
    (level1, level2, level3, leaf)
    for level1, level2_dict in database.items()
    for level2, level3_dict in level2_dict.items()
    for level3, leaf in level3_dict.items()
]
print("(playlist_id, video_id, start, text)")
print(records[100])

(playlist_id, video_id, start, text)
('PLfYUBJiXbdtSgU6S_3l6pX-4hQYKNJZFU', 'CGpR2ILao5M', 294.18, 'gonna go watch them or anything all')


In [9]:
db = sqlite3.connect('fastai_yt.db')
cur = db.cursor()

In [10]:
# virtual table configured to allow full-text search
cur.execute('DROP TABLE IF EXISTS transcriptions_fts;') 
cur.execute('CREATE VIRTUAL TABLE transcriptions_fts USING fts5(playlist_id, video_id, start, text, tokenize="porter unicode61");')

# dimension like tables
cur.execute('DROP TABLE IF EXISTS playlist;')
cur.execute('CREATE TABLE playlist (playlist_id, playlist_name);')
cur.execute('DROP TABLE IF EXISTS video;')
cur.execute('CREATE TABLE video (video_id, video_name);')

In [11]:
# bulk index records
cur.executemany('INSERT INTO transcriptions_fts (playlist_id, video_id, start, text) values (?,?,?,?);', records)
cur.executemany('INSERT INTO playlist (playlist_id, playlist_name) values (?,?);', playlists.items())
cur.executemany('INSERT INTO video (video_id, video_name) values (?,?);', videos.items())
db.commit()


In [12]:
cur.execute('SELECT start, text FROM transcriptions_fts WHERE video_id="8SF_h3xF3cE" LIMIT 5').fetchall()

[(2.0,
  'Welcome to Practical Deep Learning for coders,\xa0\nlesson one. This is version five of this course,\xa0\xa0'),
 (11.44,
  "and it's the first new one we've done\xa0\nin two years. So, we've got a lot of\xa0\xa0"),
 (15.12,
  "cool things to cover! It's amazing how much has\xa0\nchanged. Here is an xkcd from the end of 2015.\xa0\xa0"),
 (28.0,
  'Who here has seen xkcd comics before?\xa0\n…Pretty much everybody. Not surprising.\xa0\xa0'),
 (35.36,
  "So the basic joke here is… I'll let you\xa0\nread it, and then I'll come back to it.")]

In [13]:
def print_search_results(res):
    for each in res:
        print()
        print(playlists[each[0]], "->", videos[each[1]])
        print(f'"... {each[4]}..."')
        print('https://youtu.be/' + each[1] + "?t=" + str(int(each[2])))

def get_query(q, limit):
    search_in = 'text'
    if 'text:' in q: search_in = 'transcriptions_fts'
    query = f"""
    SELECT *, HIGHLIGHT(transcriptions_fts, 3, '[', ']')
    FROM transcriptions_fts WHERE {search_in} MATCH '{q}' ORDER BY rank
    LIMIT "{limit}" 
    """
    print(query)
    return query

In [ ]:
q = 'fastc*'
res = cur.execute(get_query(q, limit=5)).fetchall()
print_search_results(res)

In [ ]:
q = 'deleg*'
res = cur.execute(get_query(q, limit=5)).fetchall()
print_search_results(res)

In [ ]:
playlists

In [17]:
pl_lst = list(playlists.keys())

In [ ]:
# Search in playlist 'Practical Deep Learning for Coders 2022' or
# 'fast.ai live coding & tutorials'
q = f"""
(text: fastcore OR paral*) AND 
(playlist_id: "{pl_lst[1]}" OR "{pl_lst[2]}")
"""
res = cur.execute(get_query(q, limit=10)).fetchall()

print_search_results(res)

Building an App from the database generated.

In [ ]:
!pip install streamlit

In [20]:
pip install --upgrade protobuf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 409 kB 4.2 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.12 which is incompatible.
tensorflow-metadata 1.12.0 requires protobuf<4,>=3.13, but you have protobuf 4.21.12 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.12 which is incompatible.
streamlit 1.16.0 requires protobuf<4,>=3.12, but you have protobuf 4.21.12 which is incompatible.


In [21]:
#|export
import streamlit as st
import sqlite3

In [22]:
#| export
db_path = '/mnt/m/datamatica/posts/full-text-search-fastai-youtube-channel/'

try:
    db = sqlite3.connect(db_path + 'fastai_yt.db')
except:
    db = sqlite3.connect('fastai_yt.db')

cur = db.cursor()

In [23]:
#| export
playlist = cur.execute('SELECT playlist_id, playlist_name FROM playlist').fetchall()
video = cur.execute('SELECT video_id, video_name FROM video').fetchall()
playlist = {p: n for p, n in playlist}
video = {p: n for p, n in video}
pl_sel = list(playlist.values())
pl_to_id = {v:k for k,v in playlist.items()}

In [ ]:
#| export
st.title('Full-Text Search fastai Youtube Playlists')

# https://discuss.streamlit.io/t/select-all-on-a-streamlit-multiselect/9799/2

all_options = st.checkbox("Select all Playlists",
    key='sel_all', value=True)

container = st.container()
if all_options:
    sel_options = container.multiselect("Select one or more Playlist(s):", 
                                        pl_sel, disabled=True)
else:
    sel_options = container.multiselect("Select one or more Playlist(s):", 
                                        pl_sel, pl_sel)

if all_options: options = list(playlist.values())
else: options = sel_options

st.write('Selected playlist(s):', options)

In [29]:
#| export
def get_query(q, limit):
    
    search_in = 'text'
    
    if not( len(options)==len(pl_sel) or len(options)==0 ):
        search_in = 'transcriptions_fts'
        q_pl = '(playlist_id: '
        for pl in options:
            end = ' OR ' if pl != options[-1] else ')'
            q_pl = q_pl + f'"{pl_to_id[pl]}"' + end
        
        q = f"(text: {q}) AND " + q_pl

    query = f"""
    SELECT *, HIGHLIGHT(transcriptions_fts, 3, '[', ']')
    FROM transcriptions_fts WHERE {search_in} MATCH '{q}' ORDER BY rank
    LIMIT "{limit}" 
    """
    return query

with st.form("Input"):
    queryText = st.text_area("Search query: \ne.g. «fastc*», «fastcore OR paral*»", height=1, max_chars=None)
    limit_val = st.slider("Number of results:", min_value=5, max_value=20)
    btnResult = st.form_submit_button('Search')
    
if btnResult:
    if not queryText:
        st.text('Please enter a search query.')
    else:
        try:
            st.text('Search query generated:')
            # run query
            st.write(get_query(queryText, limit_val).replace('*', '\*'))
            res = cur.execute(get_query(q=queryText, limit=limit_val)).fetchall()
            st.text('Search results (click to go to YouTube):')

            res_md = ('  \n  '.join(['  \n  '.join([
                f"{i}.- Playlist: `{playlist[each[0]]}`, Video: `{video[each[1]]}`", 
                f"Caption: '...[{each[4].replace('[','**').replace(']','**')}](https://youtu.be/{each[1]}?t={str(int(each[2]))})...'", '\n'])
                for i, each in enumerate(res)
            ]))

            st.markdown(res_md)
        except:
            st.text('Invalid search query.')

In [ ]:
!pip install nbdev

In [39]:
#| hide
from nbdev.export import nb_export
nb_export('_Deploy_Search_Engine_Streamlit.ipynb', lib_path='/', name='app')

In [36]:
pwd

'/content'

For making a website

In [ ]:
!pip install pyngrok

In [26]:
ls

fastai_yt.db  sample_data/


References :

1. https://blog.jcharistech.com/2020/08/16/how-to-run-streamlit-apps-from-googles-colab/

2. https://fmussari.github.io/datamatica/posts/full-text-search-fastai-youtube-channel/Full-Text-Search-fastai-Youtube-Playlists-SQLite-FTS5-Part2.html